In [1]:

import ase
import numpy as np
from ase.visualize import view
from ase.io import read, write
from ase.io import cif, vasp, lammpsdata
from ase.build import make_supercell, sort, add_adsorbate, add_vacuum
from ase.build import bulk
import nglview

In [2]:
#second_slab = supercell.copy()

In [3]:
#trans = np.zeros_like(supercell.positions)
#trans[:,2] += (20 + supercell.cell[2][2])

In [4]:
#second_slab.set_positions(second_slab.positions + trans)

In [5]:
#combined_slabs = supercell + second_slab

In [6]:
#combined_slabs.cell[2][2] += (20 +supercell.cell[2][2])

In [7]:
NiPrimitive = bulk('Ni', 'fcc', a=3.528, orthorhombic=True)
NiPrimitive.cell

Cell([2.4946727240261395, 2.4946727240261395, 3.528])

In [8]:
NiPrimitive_a_b = NiPrimitive.cell[0][0]
NiPrimitive_c = NiPrimitive.cell[2][2]

In [9]:
v = view(NiPrimitive, viewer='ngl')
v

In [10]:
target_height  = 101.8
repeats_ni_a_b =  round(target_height/NiPrimitive_a_b)
repeats_ni_a_b

41

In [11]:
target_width_ni = 40
repeats_ni_c = 2*round(target_width_ni/NiPrimitive_c)
repeats_ni_c

22

In [12]:
AlPrimitive = bulk('Al', 'fcc', a=4.064, orthorhombic=True)
AlPrimitive.cell

Cell([2.873681958742129, 2.873681958742129, 4.064])

In [13]:
AlPrimitive_a_b = AlPrimitive.cell[0][0]
AlPrimitive_c = AlPrimitive.cell[2][2]

In [14]:
repeats_al_a_b =  round(target_height/AlPrimitive_a_b)
repeats_al_a_b

35

In [15]:
target_width_al = 59
repeats_al_c = 2*round(target_width_al/AlPrimitive_c)
repeats_al_c

30

In [16]:
nisalb = ase.build.fcc100("Ni", (repeats_ni_a_b, repeats_ni_a_b, repeats_ni_c), a=3.528, vacuum=1.0, orthogonal=True, periodic=True)

In [17]:
nisalb.cell

Cell([102.28158168507174, 102.28158168507174, 39.04399999999999])

In [18]:
v = view(nisalb, viewer='ngl')
v

In [19]:
alsalb = ase.build.fcc100("Al", (repeats_al_a_b, repeats_al_a_b, repeats_al_c), a=4.064, vacuum=1.0, orthogonal=True, periodic=True)

In [20]:
alsalb.cell

Cell([100.57886855597452, 100.57886855597452, 60.928])

In [21]:
v = view(alsalb, viewer='ngl')
v

In [22]:
nisalb_half  = ase.build.fcc100("Ni", (repeats_ni_a_b, repeats_ni_a_b, repeats_ni_c//2), a=3.528, vacuum=1.0, orthogonal=True, periodic=True)

In [23]:
nisalb_half.cell

Cell([102.28158168507174, 102.28158168507174, 19.64])

In [24]:
v = view(nisalb_half, viewer='ngl')
v

In [25]:
second_niaslb_half = nisalb_half.copy()

In [26]:
trans = np.zeros_like(second_niaslb_half.positions)
trans[:,2] += (20 + second_niaslb_half.cell[2][2])
second_niaslb_half.set_positions(second_niaslb_half.positions + trans)
combined_slabs = second_niaslb_half + nisalb_half
combined_slabs.cell[2][2] += (20 +nisalb_half.cell[2][2])

In [27]:
v = view(combined_slabs, viewer='ngl')
v

In [28]:
wafer = ase.build.stack(nisalb,alsalb, maxstrain=None)
cap_left = ase.build.stack(nisalb_half,alsalb, maxstrain=None)

In [29]:
reverse_wafer = ase.build.stack(alsalb,nisalb, maxstrain=None)
cap_right = ase.build.stack(alsalb,nisalb_half, maxstrain=None)


In [30]:
v = view(wafer, viewer='ngl')
v

In [31]:
v = view(reverse_wafer, viewer='ngl')
v

In [32]:
wafer.cell

Cell([101.43022512052313, 101.43022512052313, 99.97199999999998])

In [33]:
alsalb.cell

Cell([100.57886855597452, 100.57886855597452, 60.928])

In [34]:
nisalb.cell

Cell([102.28158168507174, 102.28158168507174, 39.04399999999999])

In [35]:
combined_wafer_left = ase.build.stack(wafer,wafer, maxstrain=None)
combined_wafer_left = ase.build.stack(cap_left,combined_wafer_left, maxstrain=None)

In [36]:
v = view(combined_wafer_left, viewer='ngl')
v

In [37]:
combined_wafer_right = ase.build.stack(reverse_wafer,reverse_wafer, maxstrain=None)
combined_wafer_right = ase.build.stack(combined_wafer_right,cap_right, maxstrain=None)

In [38]:
v = view(combined_wafer_right, viewer='ngl')
v

In [39]:
combined_including_al2O3 = ase.build.stack(combined_wafer_left,combined_slabs, maxstrain=None)


In [40]:
v = view(combined_including_al2O3, viewer='ngl')
v

In [41]:
combined_including_al2O3_both_sides = ase.build.stack(combined_including_al2O3,combined_wafer_right, maxstrain=None)


In [42]:
v = view(combined_including_al2O3_both_sides, viewer='ngl')
v

In [49]:
combined_including_al2O3_both_sides.write("full_slab.xyz")
combined_including_al2O3_both_sides.write("full_slab.pdb")

In [44]:
lammpsdata.write_lammps_data("full_slab.data", combined_including_al2O3_both_sides)

In [45]:
formula = combined_including_al2O3_both_sides.get_chemical_formula()
formula

'Al220500Ni221892'

In [46]:
 import re
numbers = re.findall(r'\d+', formula)


In [47]:
Al = int(numbers[0])
Ni = int(numbers[1])

In [48]:
Al/Ni

0.9937266778432751